In [2]:
import numpy as np
import matplotlib.pyplot as plt
import trimesh as tri

from utils import mesh_processing

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
mesh = tri.load_mesh("./data/test_data/bunny.ply")

In [4]:
ref_point = mesh.triangles_center[0]
dual_mesh = mesh.triangles_center
true_triangles = mesh.triangles

In [5]:
dual_mesh.shape

(69451, 3)

In [6]:
true_triangles.shape

(69451, 3, 3)

In [7]:
def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argpartition(dist_2, 3)[:3]

In [8]:
recons_triangles = np.zeros((true_triangles.shape))

In [13]:
recons_triangles = np.array([closest_node(node, mesh.vertices) for node in dual_mesh[:10]])

In [14]:
recons_triangles

array([[20462, 20463, 19669],
       [14299,  8935,  8845],
       [ 4984, 12949, 15446],
       [15447, 15446,  4984],
       [20172, 20171, 20254],
       [22324, 14686, 22609],
       [29534, 29483, 26417],
       [ 7016,  7018, 20707],
       [12255, 12343, 12613],
       [12317,  4490, 16344]])

In [18]:
print(mesh.vertices[20462])
print(mesh.vertices[20463])
print(mesh.vertices[19669])

[-0.0921802  0.132348   0.0172238]
[-0.0923144  0.132364   0.0182222]
[-0.0927674  0.130992   0.0172311]


In [19]:
true_triangles[0]

array([[-0.0927674,  0.130992 ,  0.0172311],
       [-0.0921802,  0.132348 ,  0.0172238],
       [-0.0923144,  0.132364 ,  0.0182222]])

In [44]:
closest_node(ref_point, mesh.vertices)

array([20462, 20463, 19669])

In [47]:
np.linalg.norm(ref_point - mesh.vertices[19669])

0.0010269673764337235

In [ ]:
def find_closest_triplet(point):